In [9]:
from pyspark.sql.functions import year, quarter, month, dayofmonth, hour, minute, concat, lpad ,date_format,col, lit, floor, ltrim, when, weekofyear,col, to_date
from datetime import datetime, timedelta
from pyspark.sql.types import TimestampType


StatementMeta(, 03b7214f-a56e-4249-b947-365e96ef726d, 11, Finished, Available)

# Create dim_date


In [25]:
# Define start and end dates
start_date = datetime(2014, 1, 1)
end_date = datetime(2024, 12, 31)

# Generate list of dates
date_list = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]

# Create DataFrame with dates
timestamp = spark.createDataFrame([(date,) for date in date_list], ["date_time"])

# Create dimension table for dates
dim_date = timestamp \
    .withColumn("date",date_format("date_time", 'dd/MM/yyyy') )\
    .withColumn("year", year("date_time")) \
    .withColumn("quarter", quarter("date_time")) \
    .withColumn("month", month("date_time")) \
    .withColumn("day", dayofmonth("date_time")) \
    .withColumn("day_of_week", date_format("date_time", "E")) \
    .withColumn("week_of_year", weekofyear("date_time"))\
    .withColumn("date_key", concat("year", lpad("month", 2, "0"), lpad("day", 2, "0"))) \
    .select("date_key", "date","year", "quarter", "month", "day", "day_of_week", "week_of_year")

StatementMeta(, 03b7214f-a56e-4249-b947-365e96ef726d, 27, Finished, Available)

In [20]:
display(dim_date)

StatementMeta(, 8838da56-4dab-4043-857f-6d8d16ced2de, 22, Finished, Available)

SynapseWidget(Synapse.DataFrame, 0b46c9f4-76bf-47de-b66b-c65d57eb0bcd)

# Create dim_time

In [21]:

# Tạo dữ liệu cho bảng dim_time
hours = [str(i).zfill(2) for i in range(24)]
minutes = [str(i).zfill(2) for i in range(0, 60)]  # Bắt đầu từ 1 để thể hiện từ 00:01 đến 00:60

# Tạo DataFrame chứa thông tin về giờ và phút
dim_time = spark.createDataFrame([(hour + minute, hour, minute) for hour in hours for minute in minutes], ["time_key", "hour", "minute"])

# Tạo cột time từ cột hour và minute
dim_time = dim_time.withColumn("hour", col("hour").cast("int"))  # Chuyển cột hour sang kiểu int
dim_time = dim_time.withColumn("hour", when(col("hour") < 10, col("hour").cast("string")).otherwise(col("hour")))  # Chuyển giờ dạng số < 10 thành chuỗi
dim_time = dim_time.withColumn("time", concat(col("hour"), lit(":"), lpad(col("minute"), 2, "0")))
dim_time = dim_time.withColumn("block", floor((col("hour") * 60 + col("minute").cast("int")) / 15) + 1)


StatementMeta(, 8838da56-4dab-4043-857f-6d8d16ced2de, 23, Finished, Available)

In [26]:
dim_date = dim_date.withColumn('date_key', col('date_key').cast('int'))
dim_date = dim_date.withColumn("date", to_date(col("date"), "dd/MM/yyyy"))
dim_time = dim_time.withColumn('time_key', col('time_key').cast('int'))

StatementMeta(, 03b7214f-a56e-4249-b947-365e96ef726d, 28, Finished, Available)

# Load to gold


In [27]:
dim_date.write.mode("overwrite").format("delta").saveAsTable("dim_date")

StatementMeta(, 03b7214f-a56e-4249-b947-365e96ef726d, 29, Finished, Available)

In [25]:
dim_time.write.mode("overwrite").format("delta").saveAsTable("dim_time")

StatementMeta(, 8838da56-4dab-4043-857f-6d8d16ced2de, 27, Finished, Available)